In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import numpy as np

# SQLite Database for Financial Analysis - SQLite and DB Browser 

<!-- PELICAN_BEGIN_SUMMARY -->
After merging and reshaping the datasets, we can use SQLite to store, organize and manipulate data in smaller environments.
<br>SQLite is a public-domain software package that provides a relational database management system. 
<!-- PELICAN_END_SUMMARY -->

<br>
<br>
<img src="/images/SQLflowchart.png" alt="[img: SQL flowchart]" title="SQLite flowchart" />

<br>
**Goals: Manipulate Data in SQLite**
    - Install Scientific Python "sqlite3" to convert Excel file into SQLite database in 8 lines of code
    - Reading results into a pandas DataFrame
    - Set index and modifying primary key in SQLite or in Python
    - Create query result table in SQLite and Excel
    - Creating and altering tables with pandas
    - Insert columns to the tables        
    - Use free graphical user interface (GUI) tools to run SQL queries on database
    - Repeat the process when new data is available for rerun 
    - Changes make in Excel, will automatically update in SQLite after reconnect
    - This is very helpful, no need to save and reopen Excel file
    - Note: Excel tab name cannot have spaces, but underscores are acceptable        
    

<br>
**Connecting SQLite**
    - The file file is World Population and Life Expectany by Country"
    - The second file is "Premium/Loss" with three tabs - premium, loss and reinsurance
    - The first row in each sheet should have legal column names.
    - Here is the conversion codes:
        - Import library sqlite3
        - Name of Excel .xlsx file. SQLite database will have the same name and extension .db (filename+".db")
        - Use if_exists so we can repeat the process

<br>
**First Example file "1960-2017 world population and life expectancy by country"**<br>-is from https://data.worldbank.org/indicator/SP.POP.TOTL and https://data.worldbank.org/indicator/SP.DYN.LE00.IN. 

<br>
** Convert Excel to SQLite Database**
    - Connect SQlite file
    - Changes make in Excel, will automatically update in SQLite after reconnect
    - This is very helpful, no need to save and reopen Excel file

**Connect Excel file into SQlite**<br> - The Excel file "PopLife.xlsx" has two tabs "Pop" and "Life" 

In [2]:
filename="data/SQLite/PopLife" 
con=sqlite3.connect(filename+".db")  

wb=pd.read_excel(filename+'.xlsx',sheet_name=None)
for sheet in wb:
    wb[sheet].to_sql(sheet,con, index=False, if_exists='replace')
   
con.commit()
#con.close()

<br>
**Set Index** <br> - The benefit of SQlite is we can set up unique identifer as primay key

In [3]:
## Read the first tab and set index
population = pd.read_sql('select * from Pop', con, index_col="CountryName")
print("There are %s countries - here are the first 2:" % (population.shape[0])) 
population.head(2)

There are 264 countries - here are the first 2:


,CountryCode,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
CountryName,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,...,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0
Afghanistan,AFG,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,10152331.0,10372630.0,10604346.0,...,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0


In [4]:
## Read the second tab and set index
life = pd.read_sql('select * from Life', con, index_col="CountryName")
life.head(2)

,CountryCode,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
CountryName,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,65.662,66.074,66.444,66.787,67.113,67.435,67.762,68.095,68.436,...,74.725,74.872,75.016,75.158,75.299,75.440,75.582,75.725,75.867,None
Afghanistan,AFG,32.292,32.742,33.185,33.624,34.060,34.495,34.928,35.361,35.796,...,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673,None


<br>
**Join Tables in Python then reading results into a pandas dataFrame**
    - Join tables and save the result to a third table in SQLite
    - We can save the third table as Excel too

<br>
**Merge both tabs - Population and Life Expectancy**

In [5]:
df_pop = pd.read_excel('data/SQLite/PopLife.xlsx', sheet_name = "Pop") 
df_life = pd.read_excel('data/SQLite/PopLife.xlsx', sheet_name = "Life") 
df_merge = pd.merge(df_pop, df_life, on='CountryName')
print("There are %s policies with incurred losses - here are the first 5:" % (df_merge.shape[0]))  
df_merge.head(5)

There are 264 policies with incurred losses - here are the first 5:


,CountryName,CountryCode_x,1960_x,1961_x,1962_x,1963_x,1964_x,1965_x,1966_x,1967_x,...,2008_y,2009_y,2010_y,2011_y,2012_y,2013_y,2014_y,2015_y,2016_y,2017_y
0,Aruba,ABW,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,...,74.725,74.872,75.016,75.158,75.299,75.440,75.582,75.725,75.867,NaN
1,Afghanistan,AFG,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,10152331.0,10372630.0,...,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673,NaN
2,Angola,AGO,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,6309770.0,6414995.0,...,56.189,57.231,58.192,59.042,59.770,60.373,60.858,61.241,61.547,NaN
3,Albania,ALB,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,...,75.943,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345,NaN
4,Andorra,AND,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>
**Save merge result as Excel file**

In [6]:
df_merge.to_excel('data/SQLite/PopLifeMerge.xlsx') 

<br>
**Save merge result as SQL file**

In [7]:
df_merge.to_sql("PopLifeMerge",con, if_exists='replace')
con.commit()
#con.close()

<br>
**DB Browser - a tool to visualize and access SQLite databases**
    - DB Browser for SQLite is a high quality, visual, open source tool used to create, design, and edit database files compatible with SQLite.
    - Save the query result to SQLite
    - Below is the screenshot of the database structures after connecting from Excel to SQLite by using Python  
<img src="/images/MergeSQL.png" alt="[img:DB Browser after save merge result as SQL file]" title="DbBrowser Merge" />

<br>
**Inserting a new column called "testing" to the SQlite **

In [8]:
con = sqlite3.connect("data/SQLite/PopLife.db")
cur = con.cursor()
addColumn = "ALTER TABLE pop ADD COLUMN testing varchar(32)"
cur.execute(addColumn)

In [10]:
# Rename the SQLite Table
renameTable = "ALTER TABLE tranport RENAME TO transport"
cur.execute(renameTable)

<br>
**2nd Example File - premium and loss data**

In [11]:
filename="data/SQlite/PremiumLossSQLite"   

con=sqlite3.connect(filename+".db") 
wb=pd.read_excel(filename+'.xlsx',sheet_name=None)
for sheet in wb:
    wb[sheet].to_sql(sheet,con, index=False, if_exists='replace')  
con.commit()

#con.close()    # keep the connection open until you finish all the modification

<br>
**Query the database**
    - We can repeat/automate the query process when new data get updated 
    - Save the query result to SQLite or Excel
    - Option A : Inner join between the reinsurance, premium and loss tables 
    - Option B : Selecting row

<br>
**Option A : Using SQL for Inner Ioin between the reinsurance, premium and loss tables**

In [12]:
sql_statement='''
select
r.PolicyNumber as Policy_from_Reinsurance_Tab,
p.PolicyNumber as Policy_from_Premium_Tab,
l.PolicyNumber as Policy_from_Loss_Tab
from reinsurance as r
join premium as p on p.PolicyNumber=r.PolicyNumber
join loss as l on l.PolicyNumber=r.PolicyNumber
'''
df = pd.read_sql(sql_statement,con)

df.to_sql('/SQlite/reinsurace_query_results', con,if_exists='replace') 
df.to_excel('data/SQlite/reinsurace_query_results.xlsx') 

<br>
**Option B : Using Pandas for Inner Join between the reinsurance, premium and loss tables**

In [13]:
sql_statement='select * from reinsurance'
df_reinsurance = pd.read_sql(sql_statement,con)
df_reinsurance['PolicyNumber_from_Reinsurance_Tab']=df_reinsurance['PolicyNumber']

sql_statement='select * from premium'
df_premium = pd.read_sql(sql_statement,con)
df_premium['PolicyNumber_from_Premium_Tab']=df_premium['PolicyNumber']

sql_statement='select * from loss'
df_loss = pd.read_sql(sql_statement,con)
df_loss['PolicyNumber_from_Loss_Tab']=df_loss['PolicyNumber']

df_merge = pd.merge(df_reinsurance, df_premium, on=['PolicyNumber'], how='inner', suffixes=['_from_Reinsurance_Tab','_from_Policy_Tab'])

df_merge = pd.merge(df_merge, df_loss, on=['PolicyNumber'], how='inner', suffixes=['','_from_Loss_Tab'])

df_merge[['PolicyNumber_from_Reinsurance_Tab','PolicyNumber_from_Premium_Tab','PolicyNumber_from_Loss_Tab']].head()

df_merge=df_merge[['PolicyNumber_from_Reinsurance_Tab','PolicyNumber_from_Premium_Tab','PolicyNumber_from_Loss_Tab']]

print("rows=%s" % df_merge.shape[0])

df_merge.to_sql('/SQlite/reinsurace_query_results', con,if_exists='replace') 
df_merge.to_excel('data/SQlite/reinsurace_query_results.xlsx') 


,PolicyNumber_from_Reinsurance_Tab,PolicyNumber_from_Premium_Tab,PolicyNumber_from_Loss_Tab
0,10244,10244,10244
1,10244,10244,10244
2,10509,10509,10509
3,10509,10509,10509
4,10700,10700,10700


rows=16


<br>
**Save the query result to Excel**
<img src="/images/reinsurance.png" alt="[img: Reinsurance Query Save in Excel from SQLite]" title="Reinsurance Query Excel" />

<br>
**Query Data in Python by issuing sql commands**

In [14]:
sql_statement='select * from reinsurance'
df_reinsurance = pd.read_sql(sql_statement,con)

df_reinsurance[df_reinsurance['LOB']=="Hospital"][['PolicyNumber','LOB']].head()

,PolicyNumber,LOB
44,47653,Hospital
78,46048,Hospital
88,43112,Hospital
99,37955,Hospital
115,43242,Hospital


<br>
**Insert a column to table**
    - Option A - use DML to add a column :-|
    - Option B - use to_sql to add a column - much easier and more native to PANDAS way of doing things

<br>
** View table before inserting a new column**

In [15]:
sql_statement='''
select p.* from premium as p
'''
df = pd.read_sql(sql_statement,con)
df.head()

,PolicyNumber,FirstName,LastName,PolicyEffectiveDate,LOB,PolicyType,Premium,Coverage,ExcessCoverage,ExcessPolicy
0,10880,Kurtis,Dumm,2017-07-08 00:00:00,Phy,CM,232058,1000000,3900000,96289.0
1,10948,Florencia,Bilyk,2017-06-05 00:00:00,Podiatrist,CM,5132,1000000,3900000,99671.0
2,10962,Taisha,Whack,2017-09-19 00:00:00,Dentist,OCC,11308,1000000,3900000,91809.0
3,11028,Yun,Linely,2017-10-11 00:00:00,Dentist,OCC,13381,1000000,3900000,94102.0
4,10244,Wonda,Hallsworth,2017-08-07 00:00:00,Dentist,OCC,13330,1000000,3900000,92319.0


<br>
**Option A - use DML to add a column :-|**

In [16]:
# Obtain a Cursor object to execute SQL statements

cur=con.cursor()

# Add a new column to premium table

addColumn = "ALTER TABLE premium ADD COLUMN Reinsurers varchar(32)"

cur.execute(addColumn) 

sql_statement='''
select p.* from premium as p
'''
df = pd.read_sql(sql_statement,con)
df.head()

,PolicyNumber,FirstName,LastName,PolicyEffectiveDate,LOB,PolicyType,Premium,Coverage,ExcessCoverage,ExcessPolicy,Reinsurers
0,10880,Kurtis,Dumm,2017-07-08 00:00:00,Phy,CM,232058,1000000,3900000,96289.0,None
1,10948,Florencia,Bilyk,2017-06-05 00:00:00,Podiatrist,CM,5132,1000000,3900000,99671.0,None
2,10962,Taisha,Whack,2017-09-19 00:00:00,Dentist,OCC,11308,1000000,3900000,91809.0,None
3,11028,Yun,Linely,2017-10-11 00:00:00,Dentist,OCC,13381,1000000,3900000,94102.0,None
4,10244,Wonda,Hallsworth,2017-08-07 00:00:00,Dentist,OCC,13330,1000000,3900000,92319.0,None


<br>
**Option B - use to_sql to add a column - much easier and more native to PANDAS way of doing things**

In [17]:
df['Reinsurers2']='ABC Re'
df.to_sql('premium',con,if_exists='replace') 

df.head()

,PolicyNumber,FirstName,LastName,PolicyEffectiveDate,LOB,PolicyType,Premium,Coverage,ExcessCoverage,ExcessPolicy,Reinsurers,Reinsurers2
0,10880,Kurtis,Dumm,2017-07-08 00:00:00,Phy,CM,232058,1000000,3900000,96289.0,None,ABC Re
1,10948,Florencia,Bilyk,2017-06-05 00:00:00,Podiatrist,CM,5132,1000000,3900000,99671.0,None,ABC Re
2,10962,Taisha,Whack,2017-09-19 00:00:00,Dentist,OCC,11308,1000000,3900000,91809.0,None,ABC Re
3,11028,Yun,Linely,2017-10-11 00:00:00,Dentist,OCC,13381,1000000,3900000,94102.0,None,ABC Re
4,10244,Wonda,Hallsworth,2017-08-07 00:00:00,Dentist,OCC,13330,1000000,3900000,92319.0,None,ABC Re


In [ ]:
# Closing the connection to the database file
con.close()